In [16]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

In [17]:
df1= pd.read_csv('train_LZdllcl.csv')
df2= pd.read_csv('test_2umaH9m.csv')

In [18]:
df1["education"].replace(to_replace  =np.nan, value = "No degrees", inplace=True)
df1['previous_year_rating'] = df1.ffill(axis =0)
df2["education"].replace(to_replace  =np.nan, value = "No degrees", inplace=True)
df2['previous_year_rating'] = df2.ffill(axis =0)

In [19]:
X_train = df1.iloc[:,:-1]
Y_train = df1.iloc[:,-1]

In [20]:
X_train.set_index('employee_id', inplace = True)
df2.set_index('employee_id', inplace = True)
X_train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score
employee_id,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,65438,8,1,0,49
65141,Operations,region_22,Bachelor's,m,other,1,30,65141,4,0,0,60
7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,7513,7,0,0,50
2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,2542,10,0,0,50
48945,Technology,region_26,Bachelor's,m,other,1,45,48945,2,0,0,73


In [21]:
X_train['work_fraction'] = X_train['length_of_service'] / X_train['age']
df2['work_fraction'] = df2['length_of_service'] / df2['age']
X_train['start_year'] = X_train['age'] - X_train['length_of_service']
df2['start_year'] = df2['age'] - df2['length_of_service']

In [22]:
X_train['avg training score']=X_train['avg_training_score']*X_train['no_of_trainings']
df2['avg training score']=df2['avg_training_score']*df2['no_of_trainings']

In [23]:
X_train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,work_fraction,start_year,avg training score
employee_id,,,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,Master's & above,f,sourcing,1,35,65438,8,1,0,49,0.228571,27,49
65141,Operations,region_22,Bachelor's,m,other,1,30,65141,4,0,0,60,0.133333,26,60
7513,Sales & Marketing,region_19,Bachelor's,m,sourcing,1,34,7513,7,0,0,50,0.205882,27,50
2542,Sales & Marketing,region_23,Bachelor's,m,other,2,39,2542,10,0,0,50,0.256410,29,100
48945,Technology,region_26,Bachelor's,m,other,1,45,48945,2,0,0,73,0.044444,43,73


In [24]:
ed_dict = {'Below Secondary': 0, 'Bachelor\'s': 1, 'Master\'s & above': 2, 'No degrees':-1}
X_train['education'] = X_train['education'].map(ed_dict)
#X_train['education'] = X_train['education'].fillna(-1)

X_train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,work_fraction,start_year,avg training score
employee_id,,,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,2,f,sourcing,1,35,65438,8,1,0,49,0.228571,27,49
65141,Operations,region_22,1,m,other,1,30,65141,4,0,0,60,0.133333,26,60
7513,Sales & Marketing,region_19,1,m,sourcing,1,34,7513,7,0,0,50,0.205882,27,50
2542,Sales & Marketing,region_23,1,m,other,2,39,2542,10,0,0,50,0.256410,29,100
48945,Technology,region_26,1,m,other,1,45,48945,2,0,0,73,0.044444,43,73


In [25]:
X_train.isnull().sum()

department              0
region                  0
education               0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
previous_year_rating    0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
work_fraction           0
start_year              0
avg training score      0
dtype: int64

In [26]:
df2['education'] = df2['education'].map(ed_dict)

In [27]:
one_hot_cols_train = ['department','region', 'gender', 'recruitment_channel']
one_hot = pd.get_dummies(X_train[one_hot_cols_train])
X_train = pd.concat([X_train,one_hot], axis = 1)
X_train.head()

,department,region,education,gender,recruitment_channel,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,...,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,
65438,Sales & Marketing,region_7,2,f,sourcing,1,35,65438,8,1,...,0,0,1,0,0,1,0,0,0,1
65141,Operations,region_22,1,m,other,1,30,65141,4,0,...,0,0,0,0,0,0,1,1,0,0
7513,Sales & Marketing,region_19,1,m,sourcing,1,34,7513,7,0,...,0,0,0,0,0,0,1,0,0,1
2542,Sales & Marketing,region_23,1,m,other,2,39,2542,10,0,...,0,0,0,0,0,0,1,1,0,0
48945,Technology,region_26,1,m,other,1,45,48945,2,0,...,0,0,0,0,0,0,1,1,0,0


In [28]:
X_train.drop(['department','region', 'gender', 'recruitment_channel'], axis = 1, inplace= True)
X_train.head()

,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,work_fraction,start_year,...,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,
65438,2,1,35,65438,8,1,0,49,0.228571,27,...,0,0,1,0,0,1,0,0,0,1
65141,1,1,30,65141,4,0,0,60,0.133333,26,...,0,0,0,0,0,0,1,1,0,0
7513,1,1,34,7513,7,0,0,50,0.205882,27,...,0,0,0,0,0,0,1,0,0,1
2542,1,2,39,2542,10,0,0,50,0.256410,29,...,0,0,0,0,0,0,1,1,0,0
48945,1,1,45,48945,2,0,0,73,0.044444,43,...,0,0,0,0,0,0,1,1,0,0


In [29]:
one_hot_cols_train2 = ['department','region', 'gender', 'recruitment_channel']
one_hot2 = pd.get_dummies(df2[one_hot_cols_train2])
df2 = pd.concat([df2,one_hot2], axis = 1)
df2.drop(['department','region', 'gender', 'recruitment_channel'], axis = 1, inplace= True)
df2.head()

,education,no_of_trainings,age,previous_year_rating,length_of_service,KPIs_met >80%,awards_won?,avg_training_score,work_fraction,start_year,...,region_region_5,region_region_6,region_region_7,region_region_8,region_region_9,gender_f,gender_m,recruitment_channel_other,recruitment_channel_referred,recruitment_channel_sourcing
employee_id,,,,,,,,,,,,,,,,,,,,,
8724,1,1,24,8724,1,1,0,77,0.041667,23,...,0,0,0,0,0,0,1,0,0,1
74430,1,1,31,74430,5,0,0,51,0.161290,26,...,0,0,0,0,0,1,0,1,0,0
72255,1,1,31,72255,4,0,0,47,0.129032,27,...,0,0,0,0,0,0,1,1,0,0
38562,1,3,31,38562,9,0,0,65,0.290323,22,...,0,0,0,0,0,1,0,1,0,0
64486,1,1,30,64486,7,0,0,61,0.233333,23,...,0,0,0,0,0,0,1,0,0,1


In [30]:
from sklearn.preprocessing import MinMaxScaler
minmax = MinMaxScaler()
X_train_m = pd.DataFrame(minmax.fit_transform(X_train), index = X_train.index.values, columns = X_train.columns )
df2_m = pd.DataFrame(minmax.fit_transform(df2), index = df2.index.values , columns = df2.columns )

C:\Users\saudebna\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)
C:\Users\saudebna\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype uint8, int64, float64, object were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [31]:

modelXg1 = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, gamma=0.4,nthread=4,
                       subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)


In [32]:
scores = cross_val_score(modelXg1, X_train_m, Y_train, cv = 10)
scores

KeyboardInterrupt: 

In [80]:
modelXg1.fit(X_train_m, Y_train)
res1 = modelXg1.predict(df2_m)
result1 = pd.DataFrame(res1, index = df2.index, columns =['is_promoted'] )
result1.head()

,is_promoted
employee_id,
8724,0
74430,0
72255,0
38562,0
64486,0


In [107]:
result1.to_csv('Submission_xgNEWF.csv')

In [16]:
X_train_m.shape

(54808, 59)

In [17]:
clf = SVC(C=1500, kernel='rbf', degree=3, gamma='auto_deprecated')
clf.fit(X_train_m, Y_train)

C:\Users\saudebna\Anaconda3\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1500, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [19]:
result_svc= clf.predict(df2_m)
result1 = pd.DataFrame(result_svc, index = df2.index, columns =['is_promoted'] )
result1.to_csv('Submission_svc1500.csv')

In [34]:
df1[df1['is_promoted']==1].count()

employee_id             4668
department              4668
region                  4668
education               4668
gender                  4668
recruitment_channel     4668
no_of_trainings         4668
age                     4668
previous_year_rating    4668
length_of_service       4668
KPIs_met >80%           4668
awards_won?             4668
avg_training_score      4668
is_promoted             4668
dtype: int64

In [35]:
df1[df1['is_promoted']==0].count()

employee_id             50140
department              50140
region                  50140
education               50140
gender                  50140
recruitment_channel     50140
no_of_trainings         50140
age                     50140
previous_year_rating    50140
length_of_service       50140
KPIs_met >80%           50140
awards_won?             50140
avg_training_score      50140
is_promoted             50140
dtype: int64

In [36]:
df1[df1['is_promoted']==0].count()/df1[df1['is_promoted']==1].count()

employee_id             10.741217
department              10.741217
region                  10.741217
education               10.741217
gender                  10.741217
recruitment_channel     10.741217
no_of_trainings         10.741217
age                     10.741217
previous_year_rating    10.741217
length_of_service       10.741217
KPIs_met >80%           10.741217
awards_won?             10.741217
avg_training_score      10.741217
is_promoted             10.741217
dtype: float64

In [37]:
modelXg3 = XGBClassifier(learning_rate=0.1, n_estimators=200, max_depth=4, min_child_weight=7, gamma=0.4,nthread=4,
                       subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=3,seed=29)

In [50]:
scores = cross_val_score(modelXg3, X_train_m, Y_train, cv = 10, scoring ='f1_macro')
scores

array([0.6630216 , 0.66091474, 0.65115726, 0.66310501, 0.65850715,
       0.66840327, 0.66540621, 0.66413396, 0.66949136, 0.66526633])

In [42]:
modelXg3.fit(X_train_m, Y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=0.8, gamma=0.4,
       learning_rate=0.1, max_delta_step=0, max_depth=4,
       min_child_weight=7, missing=None, n_estimators=200, n_jobs=1,
       nthread=4, objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=3, seed=29, silent=None,
       subsample=0.8, verbosity=1)

In [43]:
result_xgb= modelXg3.predict(df2_m)
result1 = pd.DataFrame(result_xgb, index = df2.index, columns =['is_promoted'] )
result1.to_csv('Submission_xgb3.csv')

In [59]:
modelXg4 = XGBClassifier(learning_rate=0.01, n_estimators=200, max_depth=4, min_child_weight=7, gamma=0.4,nthread=4,
                       subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic',scale_pos_weight=4,seed=29)

In [61]:
scores = cross_val_score(modelXg4, X_train_m, Y_train, cv = 10, scoring ='f1_macro')
scores

array([0.72400682, 0.70815593, 0.68905398, 0.69767829, 0.68203739,
       0.71853775, 0.72400682, 0.72491982, 0.71821802, 0.70053788])

In [62]:
modelXg4.fit(X_train_m, Y_train)
result_xgb= modelXg4.predict(df2_m)
result1 = pd.DataFrame(result_xgb, index = df2.index, columns =['is_promoted'] )
result1.to_csv('Submission_xgboost4.csv')